# Boosting
부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식.  
정확도가 낮은 하나의 모델을 만들어 학습 시킨뒤, 그 모델의 예측 오류는 두 번째 모델이 보완한다. 이 두 모델을 합치면 처음보다는 정확한 모델이 만들어 진다. 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정을 반복한다.


## GradientBoosting
- 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 오차를 보정할 때 경사하강법(Gradient descent)을 사용한다.
- 얕은 트리를 많이 연결하여 각각의 트리가 데이터의 일부에 대해 예측을 잘 수행하도록 하고 그런 트리들이 모여 전체 성능을 높이는 것이 기본 아이디어.
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 않좋은 단점이 있다.

### 주요 파라미터
- Decision Tree 의 가지치기 관련 매개변수
    - 각각의 tree가 복잡한 모델이 되지 않도록 한다. 
- learning rate (학습률)
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1
- n_estimators
    - tree의 개수 지정. 많을 수록 복잡한 모델이 된다.
- n_iter_no_change, validation_fraction
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 *검증점수가 좋아 지지 않으면 훈련을 조기 종료*한다.


In [21]:
from sklearn.datasets import load_breast_cancer
cancer=load_breast_cancer()
X=cancer.data
y=cancer.target

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, stratify=y)

In [23]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

def print_metrics(y, pred, title=None):
    acc=accuracy_score(y, pred)
    recall=recall_score(y, pred)
    precision=precision_score(y, pred)
    roc_auc=roc_auc_score(y, pred)
    if title:
        print(title)
    print(f'정확도:{acc}, 재현율:{recall}, 정밀도:{precision}, AUC:{roc_auc}')

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf=GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [25]:
pred_train=gb_clf.predict(X_train)
pred_test=gb_clf.predict(X_test)
print_metrics(y_train, pred_train, title='Train')
print_metrics(y_test, pred_test, title='Test')

Train
정확도:1.0, 재현율:1.0, 정밀도:1.0, AUC:1.0
Test
정확도:0.9300699300699301, 재현율:0.9666666666666667, 정밀도:0.925531914893617, AUC:0.9172955974842768


### RandomizedSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [26]:
from sklearn.model_selection import GridSearchCV
param={
    'max_depth':[1,2,3],
    'learning_rate':[0.01, 0.1, 0.5, 1]
}
g_search=GridSearchCV(GradientBoostingClassifier(),
                      param_grid=param,
                      cv=3,
                      n_jobs=-1)

In [27]:
g_search.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [28]:
g_search.best_params_

{'learning_rate': 0.5, 'max_depth': 1}

In [29]:
import pandas as pd
df=pd.DataFrame(g_search.cv_results_)
df.sort_values('rank_test_score').head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,0.060505,0.003291,0.000665,0.00047,0.5,1,"{'learning_rate': 0.5, 'max_depth': 1}",0.971831,0.971831,0.978873,0.974178,0.00332,1
7,0.063829,0.003732,0.000333,0.00047,0.5,2,"{'learning_rate': 0.5, 'max_depth': 2}",0.971831,0.978873,0.971831,0.974178,0.00332,1
3,0.060838,0.002154,0.000333,0.00047,0.1,1,"{'learning_rate': 0.1, 'max_depth': 1}",0.971831,0.971831,0.971831,0.971831,0.00000,3


In [30]:
best_model=g_search.best_estimator_
fi = best_model.feature_importances_
pd.Series(fi, index=cancer.feature_names)

mean radius                0.000000
mean texture               0.003022
mean perimeter             0.000000
mean area                  0.000000
mean smoothness            0.000472
mean compactness           0.000048
mean concavity             0.000000
mean concave points        0.083158
mean symmetry              0.000292
mean fractal dimension     0.000030
radius error               0.000339
texture error              0.005980
perimeter error            0.000000
area error                 0.000287
smoothness error           0.000780
compactness error          0.000005
concavity error            0.000000
concave points error       0.000067
symmetry error             0.000000
fractal dimension error    0.000164
worst radius               0.000557
worst texture              0.028654
worst perimeter            0.611955
worst area                 0.012696
worst smoothness           0.003484
worst compactness          0.000000
worst concavity            0.017594
worst concave points       0

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것을 알려저 유명해짐.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공하여 성능을 높임.
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치
```
conda install -c anaconda py-xgboost
만약 jupyter notebook 실행화면에서 install 하려면
conda install -y -c anaconda py-xgboost
```



## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [32]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
pred_train=xgb.predict(X_train)
pred_test=xgb.predict(X_test)

In [35]:
print_metrics(y_train, pred_train)

정확도:1.0, 재현율:1.0, 정밀도:1.0, AUC:1.0


In [37]:
print_metrics(y_test, pred_test)

정확도:0.9370629370629371, 재현율:0.9666666666666667, 정밀도:0.9354838709677419, AUC:0.9267295597484277


In [40]:
xgb.feature_importances_

array([0.02157904, 0.03271792, 0.01040448, 0.00194371, 0.00472391,
       0.01159031, 0.0656432 , 0.04427996, 0.        , 0.00469381,
       0.00447003, 0.01679804, 0.        , 0.00949945, 0.01039589,
       0.00470213, 0.        , 0.00435737, 0.00584696, 0.00574869,
       0.02978467, 0.01942805, 0.41340327, 0.03301103, 0.02140961,
       0.00883206, 0.0482654 , 0.15095131, 0.00321814, 0.01230151],
      dtype=float32)